In [ ]:
!pip install yfinance
!pip install vectorbt
!pip install groq
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt
import pandas as pd
from groq import Groq
import uuid
from datetime import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
# 使用者輸入
asset = input("請輸入資產代碼 (如 00915.TW): ") or "00915.TW"
short_ma = int(input("請輸入短期均線天數 (如 12): ") or 12)
long_ma = int(input("請輸入長期均線天數 (如 26): ") or 26)
init_cash = float(input("請輸入初始資金 (如 1000000): ") or 1000000)
fees = float(input("請輸入交易費用率 (如 0.0022125): ") or 0.0022125)
start_date = input("請輸入回測起始日期 (如 2023-01-01): ") or "2023-01-01"
end_date = input("請輸入回測結束日期 (如 2024-12-25): ") or "2024-12-25"

# 下載數據
data = yf.download(asset, start=start_date, end=end_date, interval='1d')

# 處理多層索引
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

# 設置中文字體
plt.rcParams['font.sans-serif'] = ['SimHei']  # 支援中文顯示，SimHei 為常見中文字體
plt.rcParams['axes.unicode_minus'] = False

# 計算均線
data[f'ma{short_ma}'] = data['Close'].rolling(short_ma).mean()
data[f'ma{long_ma}'] = data['Close'].rolling(long_ma).mean()

# 交易信號
data['s_in'] = (data[f'ma{short_ma}'] > data[f'ma{long_ma}']) & (data[f'ma{short_ma}'].shift(1) < data[f'ma{long_ma}'].shift(1))  # 向上突破
data['s_out'] = (data[f'ma{short_ma}'] < data[f'ma{long_ma}']) & (data[f'ma{short_ma}'].shift(1) > data[f'ma{long_ma}'].shift(1))  # 向下突破

# 移除缺失值
data.dropna(inplace=True)

# 回測
portfolio = vbt.Portfolio.from_signals(
    data['Close'].shift(-1),
    entries=data['s_in'],
    exits=data['s_out'],
    freq='1d',
    init_cash=init_cash,
    fees=fees,
    direction="longonly"
)

# 顯示回測結果
print("\n回測統計數據：")
stats = portfolio.stats()
print(stats)

請輸入資產代碼 (如 00915.TW): 0050.TW
請輸入短期均線天數 (如 12): 12
請輸入長期均線天數 (如 26): 26
請輸入初始資金 (如 1000000): 1000000
請輸入交易費用率 (如 0.0022125): 0.001425
請輸入回測起始日期 (如 2023-01-01): 2023-01-01
請輸入回測結束日期 (如 2024-12-25): 2024-12-25


[*********************100%***********************]  1 of 1 completed



回測統計數據：
Start                         2023-02-17 00:00:00
End                           2024-12-24 00:00:00
Period                          451 days 00:00:00
Start Value                             1000000.0
End Value                          1224231.457077
Total Return [%]                        22.423146
Benchmark Return [%]                    73.460498
Max Gross Exposure [%]                      100.0
Total Fees Paid                      34685.947121
Max Drawdown [%]                        16.048517
Max Drawdown Duration           171 days 00:00:00
Total Trades                                   12
Total Closed Trades                            11
Total Open Trades                               1
Open Trade PnL                                0.0
Win Rate [%]                            45.454545
Best Trade [%]                          14.695474
Worst Trade [%]                         -5.044072
Avg Winning Trade [%]                    7.110179
Avg Losing Trade [%]                    -

In [ ]:
# Groq API 風險分析
client = Groq(api_key="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")

prompt = f"""
根據以下投資組合數據，評估風險與收益等級（低風險低報酬、低風險中等報酬、低風險高報酬、中等風險低報酬、中等風險中等報酬、中等風險高報酬、高風險低報酬、高風險中等報酬、高風險高報酬）並提供簡要分析：

- 資產：{asset}
- 策略：{short_ma}日與{long_ma}日均線交叉，僅做多
- 初始資金：{init_cash}台幣
- 交易費用：{fees*100}%
- 回測期間：{start_date} 至 {end_date}
- 統計數據：{stats.to_dict()}
請用中文回答，並盡量說明風險評估的理由。
"""

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": prompt}]
)

print("\n風險評估：")
print(response.choices[0].message.content)


風險評估：
根據提供的投資組合數據，以下是對風險與收益等級的評估：

風險等級：低風險

理由：

1. 最高漲跌幅僅16.05%，低於一般投資組合的標準（通常是25%）。
2. 最高持倉 exposure 是100%，這意味著投資組合的最大持倉比例不高，避免了高度集中風險。
3. 每次交易的損失幅度平均為-2.31%，有限度地保險了投資組合的損失。

收益等級：中等報酬

理由：

1. 累計收益率22.42%，較高，但不超過通常的市場收益率。
2. 交易勝率45.45%，意味著大部分交易都是獲利的，但仍有45.55%的交易是損失的。
3. 平均獲利交易的持倉時間約44天，平均損失交易的持倉時間約9天，表明投資組合能夠有效地止損和獲利。
4. Sharpe Ratio（Sharpe比）為0.972，表明投資組合的回報高於市場風險的料，並且部分是由于策略的優秀。

總的來說，這個投資組合採用了低風險策略，主要是因為12日與26日均線交叉的關係。但是，它的收益率還是良好的，可能是因為策略的優秀和市場情況的幫助。
